<a href="https://colab.research.google.com/github/gottalottarock/ml-intro/blob/main/practice_dl_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score

# Практикум 3, DL for images



## 1. Пример работы со свертками 


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

training_data = datasets.CIFAR10(
    root='data',
    train=True, 
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root='data',
    train=False, 
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [ ]:
from torch import nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [ ]:
import torch.optim as optim

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

In [ ]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.001
[1,   101] loss: 0.115
[1,   201] loss: 0.115
[1,   301] loss: 0.115
[2,     1] loss: 0.001
[2,   101] loss: 0.115
[2,   201] loss: 0.115
[2,   301] loss: 0.115
[3,     1] loss: 0.001
[3,   101] loss: 0.114
[3,   201] loss: 0.113
[3,   301] loss: 0.111
[4,     1] loss: 0.001
[4,   101] loss: 0.107
[4,   201] loss: 0.105
[4,   301] loss: 0.104
[5,     1] loss: 0.001
[5,   101] loss: 0.101
[5,   201] loss: 0.100
[5,   301] loss: 0.097
[6,     1] loss: 0.001
[6,   101] loss: 0.095
[6,   201] loss: 0.094
[6,   301] loss: 0.094
[7,     1] loss: 0.001
[7,   101] loss: 0.092
[7,   201] loss: 0.091
[7,   301] loss: 0.090
[8,     1] loss: 0.001
[8,   101] loss: 0.088
[8,   201] loss: 0.087
[8,   301] loss: 0.087
[9,     1] loss: 0.001
[9,   101] loss: 0.085
[9,   201] loss: 0.085
[9,   301] loss: 0.085
[10,     1] loss: 0.001
[10,   101] loss: 0.084
[10,   201] loss: 0.082
[10,   301] loss: 0.082
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 41 %


In [ ]:
# prepare to count predictions for each class
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 51.3 %
Accuracy for class car   is: 54.7 %
Accuracy for class bird  is: 14.1 %
Accuracy for class cat   is: 24.1 %
Accuracy for class deer  is: 35.2 %
Accuracy for class dog   is: 47.5 %
Accuracy for class frog  is: 46.5 %
Accuracy for class horse is: 47.4 %
Accuracy for class ship  is: 51.9 %
Accuracy for class truck is: 42.1 %


# Задание 1

Написать и обучить нейронную сеть на датасете CIFAR (5 баллов)
1. Замените сверточные слои размера 5х5 на два идущих подряд слоя размером 3х3
2. Обучите модель на GPU, 20 эпох


Дополнительно:
1. (5 баллов) Переписать код с использованием pytorch lightning (см. документацию фреймворка https://pytorch-lightning.readthedocs.io/en/latest/starter/introduction.html)

## 2. Используем VGG

In [ ]:
import torchvision.models as models

vgg11 = models.vgg11(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
from torchvision import transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

training_data = datasets.CIFAR10(
    root="data", # root is the path where the train/test data is stored
    train=True, # train specifies training or test dataset
    download=True, # download=True downloads the data from the internet if it’s not available at root
    transform=transform # transform and target_transform specify the feature and label transformations
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(training_data, batch_size=128,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test_data, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Задание 2 (5 баллов)
1. Обучитите на GPU претренированную модель VGG (минимум 5 эпох)
2. Протестируйте ее на всех данных и на каждом классе отдельно

Дополнительное задание (5 баллов):     

* Взять какой-нибудь специфичный датасет с картинками
* Взять претренированную VGG (или другу модель) и сделать transfer learning на выбранный датасет